# Caching MGA results.

In [1]:
# Use results from model.energy.

In [2]:
from pathlib import Path

import pypsa

import logging
logging.getLogger("mga").setLevel(logging.INFO)
logging.getLogger("gurobipy").setLevel(logging.WARNING)
logging.getLogger("pypsa.network.io").setLevel(logging.WARNING)
logging.getLogger("pypsa.consistency").setLevel(logging.ERROR)
logging.getLogger("linopy.io").setLevel(logging.ERROR)
logging.getLogger("linopy.model").setLevel(logging.ERROR)


# Model Energy

In [3]:
n = pypsa.Network("../networks/model-energy/model-energy.nc")

In [4]:
# Solve for cost optimality.
n.optimize(solver_name="gurobi")
n.export_to_netcdf("test.nc")
n = pypsa.Network("test.nc")

Writing continuous variables.: 100%|██████████| 11/11 [00:00<00:00, 403.16it/s]

Set parameter Username
Set parameter LicenseID to value 2681762
Academic license - for non-commercial use only - expires 2026-06-24


Read LP format model from file /private/var/folders/zg/by4_k0616s98pw41wld9475c0000gp/T/linopy-problem-vxeovrht.lp
Reading time = 0.05 seconds
obj: 64246 rows, 29206 columns, 124144 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 64246 rows, 29206 columns and 124144 nonzeros
Model fingerprint: 0x672bed5b
Coefficient statistics:
  Matrix range     [2e-04, 3e+00]
  Objective range  [1e+02, 2e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 1e+04]
Presolve removed 33548 rows and 4342 columns
Presolve time: 0.03s
Presolved: 30698 rows, 24864 columns, 86254 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.01s

Barrier statistics:
 Dense cols : 6
 AA' NZ     : 7.308e+04
 Factor NZ  : 4.839e+05 (roughly 26 MB of memory)
 Factor Ops : 7.913e

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 29206 primals, 64246 duals
Objective: 8.08e+09
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Link-ext-p-lower, Link-ext-p-upper, Store-ext-e-lower, Store-ext-e-upper, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-energy_balance, Store-energy_balance were not assigned to the network.


In [5]:
# Import MGA functionalities
# # Hack to load all functions from ../../pypsa/optimization/mga.py
import sys

sys.path.append("../../pypsa/optimization")
from types import MethodType

from mga import *

n.optimize._solve_single_direction = staticmethod(
    OptimizationAbstractMGAMixin._solve_single_direction
)

n.optimize.optimize_mga_in_multiple_directions = MethodType(
    OptimizationAbstractMGAMixin.optimize_mga_in_multiple_directions, n.optimize
)

In [6]:
directions = generate_directions_random(["wind", "solar"], 100, 0)
dimensions = {
    "wind": {"Generator": {"p_nom": {"wind": 1}}},
    "solar": {"Generator": {"p_nom": {"solar": 1}}},
}

Path("cache_test").mkdir(parents=True, exist_ok=True)

In [7]:
n.optimize._solve_single_direction(
    "test.nc",
    direction=directions.iloc[0].to_dict(),
    dimensions=dimensions,
    cache_key=hash_mga(
        n, dimensions, slack=0.02
    ),
    cache_dir="cache_test",
    snapshots=None,
    multi_investment_periods=False,
    slack=0.02,
    model_kwargs=None,
    kwargs={"solver_name": "gurobi"},
)

INFO:mga:Extended hash value for MGA optimization: d03f5d79
INFO:mga:Cache HIT: direction d45cdd8e from network cache d03f5d79


({'wind': 0.2212195821529499, 'solar': 0.975224023736123},
 wind     28075.310873
 solar    38120.607074
 dtype: float64)

In [8]:
n.optimize.optimize_mga_in_multiple_directions(
    directions=directions[:5],
    dimensions=dimensions,
    snapshots=None,
    slack=0.02,
    solver_name="gurobi",
)

INFO:mga:Caching disabled
Writing continuous variables.: 100%|██████████| 11/11 [00:00<00:00, 398.00it/s]


Set parameter Username
Set parameter LicenseID to value 2681762
Academic license - for non-commercial use only - expires 2026-06-24
Set parameter Username
Set parameter Username
Set parameter LicenseID to value 2681762
Set parameter LicenseID to value 2681762
Academic license - for non-commercial use only - expires 2026-06-24
Academic license - for non-commercial use only - expires 2026-06-24
Set parameter Username
Set parameter LicenseID to value 2681762
Academic license - for non-commercial use only - expires 2026-06-24
Read LP format model from file /private/var/folders/zg/by4_k0616s98pw41wld9475c0000gp/T/linopy-problem-w1bihus7.lp
Reading time = 0.06 seconds
obj: 64247 rows, 29206 columns, 127070 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 64247 rows, 29206 columns and 127070 nonzeros
Model fingerprint: 0x54c87d35


Writing continuous variables.: 100%|██████████| 11/11 [00:00<00:00, 411.60it/s]


Set parameter Username
Set parameter LicenseID to value 2681762
Academic license - for non-commercial use only - expires 2026-06-24
Read LP format model from file /private/var/folders/zg/by4_k0616s98pw41wld9475c0000gp/T/linopy-problem-9c2x4w2a.lp
Reading time = 0.05 seconds
obj: 64247 rows, 29206 columns, 127070 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 64247 rows, 29206 columns and 127070 nonzeros
Model fingerprint: 0xd3eec105
Coefficient statistics:
  Matrix range     [2e-04, 2e+05]
  Objective range  [2e-01, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 8e+09]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve removed 33548 rows and 4342 columns
Presolve time: 0.02s
Presolved: 30699 rows, 24864 columns, 89180 nonzeros

Concurre

(       wind     solar
 0  0.221220  0.975224
 1  0.916405  0.400251
 2 -0.463647  0.886020
 3 -0.157325  0.987547
 4  0.969825 -0.243801,
            wind         solar
 0  28075.310873  38120.607074
 1  41337.746255  22393.533509
 2  26149.421228  38021.574967
 3  26706.724213  38216.272775
 4  42028.516322  18488.386539)

In [9]:
n.optimize.optimize_mga_in_multiple_directions(
    directions=directions[:5],
    dimensions=dimensions,
    cache_dir="cache_test",
    snapshots=None,
    slack=0.02,
    solver_name="gurobi",
)

INFO:mga:Extended hash value for MGA optimization: d03f5d79
INFO:mga:Using network cache: d03f5d79


(       wind     solar
 0  0.221220  0.975224
 1  0.916405  0.400251
 2 -0.463647  0.886020
 3 -0.157325  0.987547
 4  0.969825 -0.243801,
            wind         solar
 0  28075.310873  38120.607074
 1  41337.746255  22393.533509
 2  26149.421228  38021.574967
 3  26706.724213  38216.272775
 4  42028.516322  18488.386539)

# HVDC with carriers

In [10]:
n = pypsa.Network("../networks/storage-hvdc/storage-hvdc.nc")

Index(['co2_limit'], dtype='object', name='global_constraints_i')


In [11]:
# Solve for cost optimality.
n.optimize(solver_name="gurobi")
n.export_to_netcdf("test_hvdc.nc")
n = pypsa.Network("test_hvdc.nc")

Set parameter Username
Set parameter LicenseID to value 2681762
Academic license - for non-commercial use only - expires 2026-06-24
Read LP format model from file /private/var/folders/zg/by4_k0616s98pw41wld9475c0000gp/T/linopy-problem-v6sytl9z.lp
Reading time = 0.00 seconds
obj: 1104 rows, 490 columns, 2359 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1104 rows, 490 columns and 2359 nonzeros
Model fingerprint: 0x8f142d8d
Coefficient statistics:
  Matrix range     [8e-05, 6e+00]
  Objective range  [3e-03, 3e+03]
  Bounds range     [2e+01, 1e+06]
  RHS range        [2e+00, 1e+03]
Presolve removed 543 rows and 89 columns
Presolve time: 0.00s
Presolved: 561 rows, 401 columns, 2386 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     49

INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 490 primals, 1104 duals
Objective: 1.47e+07
Solver model: available
Solver message: 2

INFO:pypsa.optimization.optimize:The shadow-prices of the constraints Generator-fix-p-lower, Generator-fix-p-upper, Generator-ext-p-lower, Generator-ext-p-upper, Line-fix-s-lower, Line-fix-s-upper, Line-ext-s-lower, Line-ext-s-upper, Link-fix-p-lower, Link-fix-p-upper, Link-ext-p-lower, Link-ext-p-upper, StorageUnit-fix-p_dispatch-lower, StorageUnit-fix-p_dispatch-upper, StorageUnit-ext-p_dispatch-lower, StorageUnit-ext-p_dispatch-upper, StorageUnit-fix-p_store-lower, StorageUnit-fix-p_store-upper, StorageUnit-ext-p_store-lower, StorageUnit-ext-p_store-upper, StorageUnit-fix-state_of_charge-lower, StorageUnit-fix-state_of_charge-upper, StorageUnit-ext-state_of_charge-lower, StorageUnit-ext-state_of_charge-upper, StorageUnit-state_of_charge_set, Kirchhoff-Voltage-Law, StorageUnit-energy_balance were no

In [12]:
# Import MGA functionalities
# # Hack to load all functions from ../../pypsa/optimization/mga.py
import sys

sys.path.append("../../pypsa/optimization")
from types import MethodType

from mga import *

n.optimize._solve_single_direction = staticmethod(
    OptimizationAbstractMGAMixin._solve_single_direction
)

n.optimize.optimize_mga_in_multiple_directions = MethodType(
    OptimizationAbstractMGAMixin.optimize_mga_in_multiple_directions, n.optimize
)

In [13]:
n.components

PyPSA Components Store
- 6 'Bus' Components
- 3 'Carrier' Components
- 12 'Generator' Components
- 6 'Load' Components
- 2 'Link' Components
- 0 'Store' Components
- 6 'StorageUnit' Components
- 6 'Line' Components
- 59 'LineType' Components
- 0 'Transformer' Components
- 14 'TransformerType' Components
- 0 'ShuntImpedance' Components
- 1 'GlobalConstraint' Components
- 0 'Shape' Components
- 2 'SubNetwork' Components

In [14]:
directions = generate_directions_random(["wind", "gas"], 100, 0)

w, g = (n.generators['carrier'] == 'wind').astype(int), (n.generators['carrier'] == 'gas').astype(int)

dimensions = {
    "wind": {"Generator": {"p_nom": w}},
    "gas": {"Generator": {"p_nom": g}},
}

Path("cache_test").mkdir(parents=True, exist_ok=True)

In [15]:
(n.generators['carrier'] == 'wind').astype(int)

name
Wind 0    1
Gas 0     0
Wind 1    1
Gas 1     0
Wind 2    1
Gas 2     0
Wind 3    1
Gas 3     0
Wind 4    1
Gas 4     0
Wind 5    1
Gas 5     0
Name: carrier, dtype: int64

In [16]:
m = n.optimize.create_model()

In [17]:
direction = directions.iloc[0].to_dict()
m.objective = -sum(
            direction[key] * n.optimize.build_linexpr_from_weights(dimensions[key], model=m)
            for key in direction.keys()
        )
m.objective

Objective:
----------
LinearExpression: -0.2212 Generator-p_nom[Wind 0] - 0 Generator-p_nom[Wind 0] - 0.2212 Generator-p_nom[Wind 1] ... -0 Generator-p_nom[Wind 5] - 0 Generator-p_nom[Gas 5] - 0.9752 Generator-p_nom[Gas 5]
Sense: min
Value: None

In [18]:
r = pypsa.Network("test_hvdc.nc")

Index(['co2_limit'], dtype='object', name='global_constraints_i')


In [19]:
hash_mga(n, dimensions, slack=0.02)

INFO:mga:Extended hash value for MGA optimization: 903278e3


'903278e3'

In [20]:
hash_mga(r, dimensions, slack=0.02)

INFO:mga:Extended hash value for MGA optimization: 903278e3


'903278e3'

In [21]:
r.remove("GlobalConstraint", "co2_limit")
hash_mga(r, dimensions, slack=0.02)

INFO:mga:Extended hash value for MGA optimization: f57a3040


'f57a3040'

In [22]:
r.add("GlobalConstraint", "co2_limit", type="primary_energy", sense="<=", constant=1e9)

Index(['co2_limit'], dtype='object')


In [23]:
hash_mga(r, dimensions, slack=0.02)

INFO:mga:Extended hash value for MGA optimization: 903278e3


'903278e3'

In [24]:
n.optimize._solve_single_direction(
    "test_hvdc.nc",
    direction=directions.iloc[7].to_dict(),
    dimensions=dimensions,
    cache_key=hash_mga(n, dimensions, slack=0.02),
    cache_dir="cache_test",
    snapshots=None,
    multi_investment_periods=False,
    slack=0.02,
    model_kwargs=None,
    kwargs={"solver_name": "gurobi"},
)

INFO:mga:Extended hash value for MGA optimization: 903278e3
Index(['co2_limit'], dtype='object', name='global_constraints_i')
INFO:mga:Cache HIT: direction a760c306 from network cache 903278e3


({'wind': -0.7204928063030914, 'gas': -0.6934624114294126},
 wind    5833.710768
 gas        0.000000
 dtype: float64)

In [25]:
n.optimize.optimize_mga_in_multiple_directions(
    directions=directions[:5],  
    dimensions=dimensions,
    snapshots=None,
    slack=0.02,
    solver_name="gurobi",
    max_parallel=1,
)

INFO:mga:Caching disabled
[Worker 67870] WARNING: The following GlobalConstraint have buses which are not defined:
Index(['co2_limit'], dtype='object', name='global_constraints_i')
[Worker 67870] WARNING: The following buses have carriers which are not defined:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67870] WARNING: The following links have carriers which are not defined:
Index(['TL 0', 'TL 1'], dtype='object', name='name')
[Worker 67870] WARNING: The following lines have carriers which are not defined:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67870] WARNING: The following lines have zero r, which could break the linear load flow:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67870] WARNING: The following sub_networks have carriers which are not defined:
Index(['0', '1'], dtype='object', name='name')


Set parameter Username
Set parameter LicenseID to value 2681762
Academic license - for non-commercial use only - expires 2026-06-24
Read LP format model from file /private/var/folders/zg/by4_k0616s98pw41wld9475c0000gp/T/linopy-problem-knynopt4.lp
Reading time = 0.00 seconds
obj: 1105 rows, 490 columns, 2597 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1105 rows, 490 columns and 2597 nonzeros
Model fingerprint: 0xed042692
Coefficient statistics:
  Matrix range     [8e-05, 3e+03]
  Objective range  [2e-01, 1e+00]
  Bounds range     [2e+01, 1e+06]
  RHS range        [2e+00, 1e+07]
Presolve removed 495 rows and 65 columns
Presolve time: 0.00s
Presolved: 610 rows, 425 columns, 2417 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     52

[Worker 67870] WARNING: The following GlobalConstraint have buses which are not defined:
Index(['co2_limit'], dtype='object', name='global_constraints_i')
[Worker 67870] WARNING: The following buses have carriers which are not defined:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67870] WARNING: The following links have carriers which are not defined:
Index(['TL 0', 'TL 1'], dtype='object', name='name')
[Worker 67870] WARNING: The following lines have carriers which are not defined:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67870] WARNING: The following lines have zero r, which could break the linear load flow:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67870] WARNING: The following sub_networks have carriers which are not defined:
Index(['0', '1'], dtype='object', name='name')


Set parameter Username
Set parameter LicenseID to value 2681762
Academic license - for non-commercial use only - expires 2026-06-24
Read LP format model from file /private/var/folders/zg/by4_k0616s98pw41wld9475c0000gp/T/linopy-problem-_0bexs1b.lp
Reading time = 0.00 seconds
obj: 1105 rows, 490 columns, 2597 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1105 rows, 490 columns and 2597 nonzeros
Model fingerprint: 0x4174e1ff
Coefficient statistics:
  Matrix range     [8e-05, 3e+03]
  Objective range  [4e-01, 9e-01]
  Bounds range     [2e+01, 1e+06]
  RHS range        [2e+00, 1e+07]
Presolve removed 495 rows and 65 columns
Presolve time: 0.00s
Presolved: 610 rows, 425 columns, 2417 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     48

[Worker 67878] WARNING: The following GlobalConstraint have buses which are not defined:
Index(['co2_limit'], dtype='object', name='global_constraints_i')
[Worker 67878] WARNING: The following buses have carriers which are not defined:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67878] WARNING: The following links have carriers which are not defined:
Index(['TL 0', 'TL 1'], dtype='object', name='name')
[Worker 67878] WARNING: The following lines have carriers which are not defined:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67878] WARNING: The following lines have zero r, which could break the linear load flow:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67878] WARNING: The following sub_networks have carriers which are not defined:
Index(['0', '1'], dtype='object', name='name')


...

[Worker 67878] WARNING: The following GlobalConstraint have buses which are not defined:
Index(['co2_limit'], dtype='object', name='global_constraints_i')
[Worker 67878] WARNING: The following buses have carriers which are not defined:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67878] WARNING: The following links have carriers which are not defined:
Index(['TL 0', 'TL 1'], dtype='object', name='name')
[Worker 67878] WARNING: The following lines have carriers which are not defined:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67878] WARNING: The following lines have zero r, which could break the linear load flow:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67878] WARNING: The following sub_networks have carriers which are not defined:
Index(['0', '1'], dtype='object', name='name')


Set parameter Username
Set parameter LicenseID to value 2681762
Academic license - for non-commercial use only - expires 2026-06-24
Read LP format model from file /private/var/folders/zg/by4_k0616s98pw41wld9475c0000gp/T/linopy-problem-ygm1x3e_.lp
Reading time = 0.00 seconds
obj: 1105 rows, 490 columns, 2597 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1105 rows, 490 columns and 2597 nonzeros
Model fingerprint: 0x86241227
Coefficient statistics:
  Matrix range     [8e-05, 3e+03]
  Objective range  [2e-01, 1e+00]
  Bounds range     [2e+01, 1e+06]
  RHS range        [2e+00, 1e+07]
Presolve removed 495 rows and 65 columns
Presolve time: 0.00s
Presolved: 610 rows, 425 columns, 2417 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     48

[Worker 67881] WARNING: The following GlobalConstraint have buses which are not defined:
Index(['co2_limit'], dtype='object', name='global_constraints_i')
[Worker 67881] WARNING: The following buses have carriers which are not defined:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67881] WARNING: The following links have carriers which are not defined:
Index(['TL 0', 'TL 1'], dtype='object', name='name')
[Worker 67881] WARNING: The following lines have carriers which are not defined:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67881] WARNING: The following lines have zero r, which could break the linear load flow:
Index(['0', '1', '2', '3', '4', '5'], dtype='object', name='name')
[Worker 67881] WARNING: The following sub_networks have carriers which are not defined:
Index(['0', '1'], dtype='object', name='name')


Set parameter Username
Set parameter LicenseID to value 2681762
Academic license - for non-commercial use only - expires 2026-06-24
Read LP format model from file /private/var/folders/zg/by4_k0616s98pw41wld9475c0000gp/T/linopy-problem-k84j_lz4.lp
Reading time = 0.00 seconds
obj: 1105 rows, 490 columns, 2597 nonzeros
Gurobi Optimizer version 11.0.3 build v11.0.3rc0 (mac64[arm] - Darwin 25.0.0 25A362)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1105 rows, 490 columns and 2597 nonzeros
Model fingerprint: 0x3fc336b9
Coefficient statistics:
  Matrix range     [8e-05, 3e+03]
  Objective range  [2e-01, 1e+00]
  Bounds range     [2e+01, 1e+06]
  RHS range        [2e+00, 1e+07]
Presolve removed 495 rows and 65 columns
Presolve time: 0.00s
Presolved: 610 rows, 425 columns, 2417 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0      handle free variables                          0s
     51

(       wind       gas
 0  0.221220  0.975224
 1  0.916405  0.400251
 2 -0.463647  0.886020
 3 -0.157325  0.987547
 4  0.969825 -0.243801,
           wind          gas
 0  5884.768466  1685.169611
 1  6114.922836  1326.411328
 2  5836.779324  1691.311760
 3  5836.779324  1691.311760
 4  6447.585397   207.307629)

In [26]:
n.optimize.optimize_mga_in_multiple_directions(
    directions=directions[:5],
    dimensions=dimensions,
    cache_dir="cache_test",
    snapshots=None,
    slack=0.02,
    solver_name="gurobi",
    max_parallel=4,
)

INFO:mga:Extended hash value for MGA optimization: 903278e3
INFO:mga:Using network cache: 903278e3
[Worker 67885] WARNING: The following GlobalConstraint have buses which are not defined:
Index(['co2_limit'], dtype='object', name='global_constraints_i')
[Worker 67887] WARNING: The following GlobalConstraint have buses which are not defined:
Index(['co2_limit'], dtype='object', name='global_constraints_i')
[Worker 67886] WARNING: The following GlobalConstraint have buses which are not defined:
Index(['co2_limit'], dtype='object', name='global_constraints_i')
[Worker 67888] WARNING: The following GlobalConstraint have buses which are not defined:
Index(['co2_limit'], dtype='object', name='global_constraints_i')
[Worker 67900] WARNING: The following GlobalConstraint have buses which are not defined:
Index(['co2_limit'], dtype='object', name='global_constraints_i')


(       wind       gas
 0  0.221220  0.975224
 1  0.916405  0.400251
 2 -0.463647  0.886020
 3 -0.157325  0.987547
 4  0.969825 -0.243801,
           wind          gas
 0  5884.768466  1685.169611
 1  6114.922836  1326.411328
 2  5836.779324  1691.311760
 3  5836.779324  1691.311760
 4  6447.585397   207.307629)